In [1]:
import cv2 as cv
import os
import numpy as np
#import tracker1 as tr
import tracker1
#rom tracker1 import Objecte as o

AttributeError: partially initialized module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline' (most likely due to a circular import)

In [2]:
! pip uninstall opencv-contrib-python
! pip uninstall opencv-python
! pip uninstall opencv-python-headless

! pip install opencv-python

^C


^C


In [2]:
pip install opencv-python==4.6.0.66

     -------------------------------------- 35.6/35.6 MB 517.1 kB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.64
    Uninstalling opencv-python-4.5.5.64:
      Successfully uninstalled opencv-python-4.5.5.64
Note: you may need to restart the kernel to use updated packages.


In [2]:
#tracker

class Tracker():
    
    def __init__(self, objectes = [], ids = [], max_frame = 10):
        self._objectes = []
        self._ids = []
        self._max_frame = max_frame


    def deteccio(self, objectes_detectats):
        
        if self._objectes == []:
            for i, new_object in enumerate(objectes_detectats):              
                id = len(self._ids) + 1
                new_object._id = id
                
                self._ids.append(id)
                self._objectes.append(new_object)
        
        
        else:
            matriu_distancies = np.zeros( (len(self._objectes), len(objectes_detectats)))
            
            for i, objecte in enumerate(self._objectes):
                for j, objecte_candidat in enumerate(objectes_detectats):
                    matriu_distancies[i,j] = objecte.dist_euclidean(objecte_candidat)
            
            
            valor_maximo = matriu_distancies.sum()
            
            if len(self._objectes) <= len(objectes_detectats):
                for i in range (len(self._objectes)):
                    ix_objectes, ix_detactats = np.where(matriu_distancies == np.min(matriu_distancies))
                    ix_objectes, ix_detactats = ix_objectes[-1], ix_detactats[-1]
                    
                    
                    self._objectes[int(ix_objectes)]._centroide = objectes_detectats[int(ix_detactats)]._centroide
                    self._objectes[int(ix_objectes)]._no_esta = 0
                    
                    matriu_distancies[ix_objectes, :] = valor_maximo
                    matriu_distancies[:, ix_detactats] = valor_maximo
                    
                for i in range(len(objectes_detectats) - len(self._objectes) ):
                    ix_objectes, ix_detactats = np.where(matriu_distancies == np.min(matriu_distancies))
                    
                    new_object = objectes_detectats[int(ix_detactats[-1])]
                    id = len(self._ids) + 1
                    new_object._id = id
                    self._objectes.append(new_object)
                    self._ids.append(id)
                    
                    
                    matriu_distancies[ix_objectes, :] = valor_maximo
                    matriu_distancies[:, ix_detactats] = valor_maximo
                
            
            
            elif len(objectes_detectats) == 0:
                for ob in self._objectes:
                    ob._no_esta += 1
                    if ob._no_esta > self._max_frame:
                        self._objectes.remove(ob)
            
            else:
                for i in range (len(objectes_detectats)):
                    ix_objectes, ix_detactats = np.where(matriu_distancies == np.min(matriu_distancies))
                    ix_objectes, ix_detactats = ix_objectes[-1], ix_detactats[-1]
                    
                    self._objectes[int(ix_objectes)]._centroide = objectes_detectats[int(ix_detactats)]._centroide
                    
                    self._objectes[int(ix_objectes)]._no_esta = 0
                    
                    matriu_distancies[ix_objectes, :] = valor_maximo
                    matriu_distancies[:, ix_detactats] = valor_maximo
                    
                
                for i in range(len(self._objectes) - len(objectes_detectats)):
                    ix_objectes, ix_detactats = np.where(matriu_distancies == np.min(matriu_distancies))
                    
                    ob = objectes_detectats[int(ix_objectes[-1])]
                    ob._no_esta += 1
                    
                    if ob._no_esta > self._max_frame:
                        self._objectes.remove(ob)
                        
                    
                    
                    matriu_distancies[ix_objectes, :] = valor_maximo
                    matriu_distancies[:, ix_detactats] = valor_maximo
                    
                    
                    
                
                       

class Contador():
    def __init__(self, y = 0) :
        self._y = y
        self._sobre = []
        self._per_sota = []
        self._contat = []
        self._contar_baixa = 0
        self._contar_puja = 0
        
    
    def contar(self, centroide, nom):
        y_actual = centroide[1]
        
        if nom not in self._contat:
            
            if nom in self._per_sota and self._y > y_actual:
                self._contar_puja += 1
                print('puja:', self._contar_puja)
                self._contat.append(nom)
                
            if nom in self._sobre and self._y < y_actual:
                self._contar_baixa += 1
                print('baiax:', self._contar_baixa)
                self._contat.append(nom)
            
            if  nom not in self._per_sota and nom not in self._sobre:
                if self._y  > y_actual:
                    self._sobre.append(nom)
                else:
                    self._per_sota.append(nom)
            
            
        
        
        
        
                
        
        

            
            
class Objecte():
    
    def __init__(self, centroide = (0,0), id = None):
        self._centroide = centroide
        self._id = id
        self._no_esta = 0
        
        
    
    def dist_euclidean(self, altre_centroide):
        dist = (self._centroide[0] - altre_centroide._centroide[0]) ** 2 + (self._centroide[1] - altre_centroide._centroide[1]) ** 2
        return dist


In [138]:
tracking = Tracker()


In [54]:
capture = cv.VideoCapture('output7.mp4')
count = 0
path = 'C:/Users/LENOVO/Documents/uni/4t/psiv/tracking/psiv_trackers/frames/'
while ( capture.isOpened()):
    ret, frame = capture.read()
    if (ret == True):
        cv.imwrite(path +  'img_%010d.jpg' % count, frame)
        count += 1
        if cv.waitKey(1) == ord('s'):
            break
    else:
        break
capture.release()
cv.destroyAllWindows()

#guardar imagene en un archivo

KeyboardInterrupt: 

In [3]:
capture = cv.VideoCapture('output7.mp4')
fgbg = cv.bgsegm.createBackgroundSubtractorMOG()

tracking = Tracker()
linia = 200
conut = Contador(linia)


while ( capture.isOpened()):
    ret, frame = capture.read()
    if (ret == False): break
    
    
    altura, ancho, _ = frame.shape
    
    
    roi = frame[600:, :550]
    
    
    fgmaskMOG = fgbg.apply(roi)
     
    tratada = cv.dilate(fgmaskMOG, np.ones((3,3)), iterations=7)
    tratada = cv.morphologyEx(tratada,cv.MORPH_OPEN, np.ones((30,30)))
    
    
    
    #detectar objeto
    objetetos_detectados = []
    contours, _ = cv.findContours(tratada, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area > 4000:
        #Calculate area remove small elements (personas/motos)
            #cv.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv.boundingRect(cnt)
            
            if 0.6 < w/h and   w/h < 1.5:
                cv.rectangle(roi, (x,y), (x + w, y + h), (0, 255, 0), 3)
                
                centroide = (int(x + w/2) ,int(y + h/2))
                
                objecte = Objecte(centroide)
                objetetos_detectados.append(objecte)
            
            tracking.deteccio(objetetos_detectados)
            
            for ob in tracking._objectes:
                
                conut.contar(centroide = ob._centroide, nom =ob._id)
                
                cv.putText(roi, str(ob._id), ob._centroide, cv.FONT_HERSHEY_SIMPLEX, 0.5, (2255, 255, 0) )
            
    
    cv.line(roi, (0,linia), (99999, linia) , (0,0,255), 10 )
    cv.imshow('roi', roi)
    cv.imshow('MOG', tratada)
    cv.imshow('frame', frame)
    
    
    if cv.waitKey(1) == ord('q'):
            break
    
    
capture.release()
cv.destroyAllWindows()


puja: 1
puja: 2
baiax: 1
baiax: 2
puja: 3
puja: 4
puja: 5
puja: 6


In [182]:
capture.release()
cv.destroyAllWindows()

In [2]:
! pip install ultralytics

^C


In [ ]:


#fourcc = cv.VideoWriter_fourcc(*'XVID')  # Puedes cambiar 'XVID' al codec que prefieras
#output_video = cv.VideoWriter('output.avi', fourcc, 20.0, (int(video_cap.get(3)), int(video_cap.get(4))))



In [1]:

import numpy as np
import datetime
from ultralytics import YOLO
model = YOLO("yolov8s.pt")
video_cap = cv.VideoCapture('output7.mp4')
conf_threshold = 0.5


while True:
    # starter time to computer the fps
    start = datetime.datetime.now()
    ret, frame = video_cap.read()

    # if there is no frame, we have reached the end of the video
    if not ret:
        print("End of the video file...")
        break
    ############################################################
    ### Detect the objects in the frame using the YOLO model ###
    ############################################################
    # run the YOLO model on the frame
    results = model(frame)
    # loop over the results
    for result in results:
      # initialize the list of bounding boxes, confidences, and class IDs
      bboxes = []
      confidences = []
      class_ids = []
      # loop over the detections
      for data in result.boxes.data.tolist():
          x1, y1, x2, y2, confidence, class_id = data
          x = int(x1)
          y = int(y1)
          w = int(x2) - int(x1)
          h = int(y2) - int(y1)
          class_id = int(class_id)
          # filter out weak predictions by ensuring the confidence is
          # greater than the minimum confidence
          if confidence > conf_threshold:
              bboxes.append([x, y, w, h])
              confidences.append(confidence)
              class_ids.append(class_id)
              cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    end = datetime.datetime.now()
    # calculate the frame per second and draw it on the frame
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv.putText(frame, fps, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)
    # show the output frame
    cv.imshow('frame', frame)
    # Escribir el frame en el video de salida
    #output_video.write(frame)

AttributeError: partially initialized module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline' (most likely due to a circular import)